In [1]:
import os 
%pwd

'd:\\VS_code\\MLops\\11_End-to-End_Projects\\End-to-End_TextSummarizer_Project\\research'

In [2]:
os.chdir("..")
%pwd

'd:\\VS_code\\MLops\\11_End-to-End_Projects\\End-to-End_TextSummarizer_Project'

## Basic Config

In [3]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass 
class DataTransformationConfig:
    root_dir: Path 
    data_path: Path 
    tokenizer_name: Path

In [4]:
from src.TextSummarizer.constants import * 
from src.TextSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH, 
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root_dir])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir, 
            data_path= config.data_path, 
            tokenizer_name= config.tokenizer_name
        )
        return data_transformation_config

In [6]:
import os 
from src.TextSummarizer.logging import logger 

from transformers import AutoTokenizer 

d:\VS_code\MLops\11_End-to-End_Projects\End-to-End_TextSummarizer_Project\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# !pip install datasets
# import datasets 

from datasets import load_from_disk

### Data Transformation Components

In [8]:
class DataTransformation:
    def __init__(self,
                 config:DataTransformationConfig):
        self.config = config 
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encoding = self.tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch["summary"], max_length=1024, truncation=True)

        return {
            "input_ids": input_encoding["input_ids"],
            "attention_mask": input_encoding["attention_mask"],
            "labels": target_encoding["input_ids"]
            }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        # save to disk 
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))


In [9]:
config = ConfigurationManager()

data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config = data_transformation_config)
data_transformation.convert()

[2025-01-21 22:55:20,506: INFO: common: yaml file config\config.yaml loaded successfully]
[2025-01-21 22:55:20,924: INFO: common: yaml file params.yaml loaded successfully]
[2025-01-21 22:55:21,044: INFO: common: Created Directory at: artifacts]
[2025-01-21 22:55:21,259: INFO: common: Created Directory at: artifacts/data_transformation]


d:\VS_code\MLops\11_End-to-End_Projects\End-to-End_TextSummarizer_Project\myenv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nisha\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\VS_c